# Editing Audio Metadata

Minim can organize your local music library by tagging audio files with metadata retrieved from popular music services, such as iTunes, Spotify, and TIDAL.

In [1]:
from pathlib import Path

import numpy as np

from minim import audio, itunes, spotify, tidal, utility

## Setup

### Instantiating API clients

In [2]:
client_itunes = itunes.SearchAPI()
client_spotify = spotify.WebAPI()
client_tidal = tidal.PrivateAPI()

### Finding audio files

In [3]:
audio_files = [f for f in (Path().resolve().parents[3] / "tests/data/previews/").glob("**/*") if f.suffix == ".flac"]

### Defining helper functions

In [4]:
def print_metadata(audio_file):
    for field, value in audio_file.__dict__.items():
        if not field.startswith("_"):
            if field in {"artwork", "lyrics"}:
                if value:
                    value = type(value)
            field = field.upper() if field == "isrc" else field.replace("_", " ").capitalize()
            print(f"{field}: {value}")

## Example 1: Tagging an audio file with no metadata

In [5]:
audio_file = audio.Audio(audio_files[0], pattern=("(.*)_(.*)", ("artist", "title")))
audio_file

In [6]:
audio_file.convert("mp3")
audio_file

size=    1033kB time=00:00:30.09 bitrate= 281.3kbits/s speed=68.8x    


In [7]:
print_metadata(audio_file)

Album: None
Album artist: None
Artist: spektrem
Comment: None
Compilation: None
Composer: None
Copyright: None
Date: None
Genre: None
ISRC: None
Lyrics: None
Tempo: None
Title: shine
Disc number: None
Disc count: None
Track number: None
Track count: None
Artwork: None
Bit depth: None
Bitrate: 281012
Channel count: 2
Codec: mp3
Sample rate: 44100


In [8]:
query = f"{audio_file.artist} {audio_file.title}".lower()
itunes_results = client_itunes.search(query)["results"]
itunes_track = itunes_results[np.argmax(utility.levenshtein_ratio(query, [f"{r['artistName']} {r['trackName']}".lower() for r in itunes_results]))]
itunes_album = client_itunes.lookup(itunes_track["collectionId"])["results"][0]
audio_file.set_metadata_using_itunes(itunes_track, album_data=itunes_album, overwrite=True)
print_metadata(audio_file)

Album: Enter the Spektrem - Single
Album artist: Spektrem
Artist: Spektrem
Comment: None
Compilation: False
Composer: None
Copyright: ℗ 2013 GFTED
Date: 2013-03-06T12:00:00Z
Genre: Electronic
ISRC: None
Lyrics: None
Tempo: None
Title: Shine
Disc number: 1
Disc count: 1
Track number: 2
Track count: 3
Artwork: <class 'bytes'>
Bit depth: None
Bitrate: 281012
Channel count: 2
Codec: mp3
Sample rate: 44100


In [9]:
spotify_results = client_spotify.search(query, type="track")["items"]
spotify_track = spotify_results[np.argmax(utility.levenshtein_ratio(query, [f"{r['artists'][0]['name']} {r['name']}".lower() for r in spotify_results]))]
audio_file.set_metadata_using_spotify(spotify_track, audio_features=client_spotify.get_track_audio_features(spotify_track["id"]))
print_metadata(audio_file)

Album: Enter the Spektrem - Single
Album artist: Spektrem
Artist: Spektrem
Comment: None
Compilation: False
Composer: None
Copyright: ℗ 2013 GFTED
Date: 2013-03-06T12:00:00Z
Genre: Electronic
ISRC: GB2LD0901581
Lyrics: None
Tempo: 128
Title: Shine
Disc number: 1
Disc count: 1
Track number: 2
Track count: 3
Artwork: <class 'bytes'>
Bit depth: None
Bitrate: 281012
Channel count: 2
Codec: mp3
Sample rate: 44100


In [10]:
tidal_results = client_tidal.search(query)["tracks"]["items"]
tidal_track = next((r for r in tidal_results if r["isrc"] == audio_file.isrc), None)
tidal_album = client_tidal.get_album(tidal_track["album"]["id"])
tidal_composers = client_tidal.get_track_composers(tidal_track["id"])
tidal_lyrics = client_tidal.get_track_lyrics(tidal_track["id"])
audio_file.set_metadata_using_tidal(tidal_track, album_data=tidal_album, composers=tidal_composers, lyrics=tidal_lyrics)
print_metadata(audio_file)

Album: Enter the Spektrem - Single
Album artist: Spektrem
Artist: Spektrem
Comment: None
Compilation: False
Composer: None
Copyright: ℗ 2013 GFTED
Date: 2013-03-06T12:00:00Z
Genre: Electronic
ISRC: GB2LD0901581
Lyrics: <class 'str'>
Tempo: 128
Title: Shine
Disc number: 1
Disc count: 1
Track number: 2
Track count: 3
Artwork: <class 'bytes'>
Bit depth: None
Bitrate: 281012
Channel count: 2
Codec: mp3
Sample rate: 44100


## Example 2: Tagging an audio file with partial metadata